<center style="font-size:30px;padding-bottom:10px;color:DarkBlue;">Gestión de Datos</center>
<center> _____________________________________________________________________________________________ </center>
<p>&nbsp;</p>
<center style="font-size:25px;padding-bottom:10px;color:CornFlowerBlue;">Práctica 4</center>
<center style="font-size:20px;padding-bottom:10px;color:LightCoral;">Ejercicio sobre Analítica y Visualización de Datos con Power BI </center>
<center style="font-size:15px;">María Barroso Honrubia</center>
<center style="font-size:15px;">Gloria del Valle Cano</center>
<center style="font-size:12px;">17 de diciembre 2021</center>


In [10]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [11]:
# Observamos los datos descargados
df = pd.read_csv('casos_diagnostico_ccaa.csv')
df.head()

,ccaa_iso,fecha,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_ag,num_casos_prueba_elisa,num_casos_prueba_desconocida
0,AN,2020-01-01,0,0,0,0,0,0
1,AR,2020-01-01,0,0,0,0,0,0
2,AS,2020-01-01,0,0,0,0,0,0
3,CB,2020-01-01,0,0,0,0,0,0
4,CE,2020-01-01,1,0,0,1,0,0


In [12]:
# Vemos la información de la primera columna
isos = sorted(list(set(list(df.ccaa_iso))))
isos

['AN',
 'AR',
 'AS',
 'CB',
 'CE',
 'CL',
 'CM',
 'CN',
 'CT',
 'EX',
 'GA',
 'IB',
 'MC',
 'MD',
 'ML',
 'NC',
 'PV',
 'RI',
 'VC']

In [5]:
# Buscamos la ISO oficial asociada a la comunidad autónoma
r = requests.get("https://es.wikipedia.org/wiki/ISO_3166-2:ES#cite_note-newsletter-1")
soup = BeautifulSoup(r.text, 'html.parser')

# Encontramos la primera tabla de Wikipedia
table = soup.find_all('table')[0]

# Obtenemos las filas en HTML
rows = table.findAll(lambda tag: tag.name=='tr')

# Obtenemos el contenido de la tabla
ccaa = [list(filter(lambda item: item, rows[iso].text.strip().split('\n'))) for iso, _ in enumerate(rows)][1:]

# Limpiamos los strings de la ISO y de los nombres
new_isos = [i.split('ES-')[1] for i in [ccaa[i][0].split(',')[0].split('(')[0].split('/')[0].strip() for i in range(len(ccaa[1:]))]]
nombres = [ccaa[i][1].split(',')[0].split('(')[0].split('/')[0].strip() for i in range(len(ccaa[1:]))]

# Guardamos en un diccionario los nombres con su correspondiente ISO
dicc = {k:v for k,v in zip(new_isos, sorted(nombres))}

In [6]:
# Vemos que faltan 3 datos
set(isos) - set(new_isos)

{'CE', 'ML', 'VC'}

In [7]:
# Los añadimos a mano
dicc['VC'] = 'Comunidad Valenciana'
dicc['CE'] = 'Ceuta'
dicc['ML'] = 'Melilla'
dicc

{'AN': 'Andalucía',
 'AR': 'Aragón',
 'AS': 'Asturias',
 'CN': 'Canarias',
 'CB': 'Cantabria',
 'CM': 'Castilla y León',
 'CL': 'Castilla-La Mancha',
 'CT': 'Catalunya',
 'EX': 'Extremadura',
 'GA': 'Galicia',
 'IB': 'Illes Balears',
 'RI': 'La Rioja',
 'MD': 'Madrid',
 'MC': 'Murcia',
 'NC': 'Navarra',
 'PV': 'País Vasco',
 'VC': 'Comunidad Valenciana',
 'CE': 'Ceuta',
 'ML': 'Melilla'}

In [13]:
# Cambiamos el identificador de la comunidad autónoma
df.ccaa_iso = df.ccaa_iso.map(dicc)

# Cambiamos el nombre de las columnas del csv
df.columns = ['Comunidad Autonoma', 
              'Fecha', 
              'Casos confirmados', 
              'Casos con test PCR', 
              'Casos con anticuerpos',
              'Casos con test antigenos',
              'Casos con test Elisa',
              'Casos con otra prueba']

In [14]:
# Vemos como ha quedado
df.head()

,Comunidad Autonoma,Fecha,Casos confirmados,Casos con test PCR,Casos con anticuerpos,Casos con test antigenos,Casos con test Elisa,Casos con otra prueba
0,Andalucía,2020-01-01,0,0,0,0,0,0
1,Aragón,2020-01-01,0,0,0,0,0,0
2,Asturias,2020-01-01,0,0,0,0,0,0
3,Cantabria,2020-01-01,0,0,0,0,0,0
4,Ceuta,2020-01-01,1,0,0,1,0,0


In [18]:
# Revisamos que no hay duplicados
df.duplicated().any()

False

In [17]:
# Revisamos que no hayan valores NaN
df.isna().any()

Comunidad Autonoma          False
Fecha                       False
Casos confirmados           False
Casos con test PCR          False
Casos con anticuerpos       False
Casos con test antigenos    False
Casos con test Elisa        False
Casos con otra prueba       False
dtype: bool

In [19]:
# Guardamos el nuevo dataset
df.to_csv('covid_processed_data.csv', index=False)

In [20]:
# Tambien en formato excel
df.to_excel('covid_processed_data.xlsx', engine='xlsxwriter')